# **Data Transformation**

*This is where we do data wrangling and EDA*

In [7]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from typing import Optional, Tuple, List

# **1. Alcohol Consumption, BMI, Dietary, Smoking and Vaping**

Based on "Alcohol_Consumption", for SQL, we need a tidy format where each row is one observation:
- `alco_sex`: gender ("Persons", "Males", "Females")
- `alco_category_group`: high-level grouping (e.g. "Exceeded guideline(e)", "Did not exceed guideline", etc.)
- `alco_category`: specific measure under that group (e.g. "Consumed more than 10 drinks in the last week", "Consumed 5 or more drinks on any day...", "Total exceeded guideline")
- `alco_age_group`: demographic column from the headers (e.g. "15–17(c)", "18–24", "25–34", "65 years and over", "Total 18 years and over")
- `alco_estimate_000`: the numeric value in the table (population count in '000s)

Similarly for all other files since they have same formats.


In [8]:
def process_estimates_file(file_path: str, prefix: str, output_path: Optional[str] = None) -> str:
    """
    Read an Excel file that contains a survey 'Estimates' sheet, clean it, reshape to tidy format,
    and write out a CSV. Returns the path to the written CSV.

    Steps:
      1) Open the first sheet whose name contains 'Estimates' (case-insensitive).
      2) Remove metadata block (from the row starting with '* estimate' through the row containing
         '© Commonwealth of Australia', case-insensitive, inclusive).
      3) Auto-detect the header (age groups) row and promote it to column headers:
         - If a row in col0 contains 'Age group', use the *next* row as the header row.
         - Else, among the first ~10 rows, pick the row with the most non-null values across cols 1:.
      4) Drop completely empty columns/rows.
      5) Strip footnote markers like '(a)'..'(h)' anywhere in the sheet contents.
      6) Identify structural section headers:
         - '<prefix>_sex' section headers: 'Persons', 'Males', 'Females' (case-insensitive).
         - '<prefix>_category_group' headers: any row with all-NaN across age columns and not a sex header.
         Forward-fill both downwards.
      7) Keep only data rows (i.e., not section headers), rename the first column to '<prefix>_category'.
      8) Melt to tidy format with columns:
         '<prefix>_sex', '<prefix>_category_group', '<prefix>_category',
         '<prefix>_age_group', '<prefix>_estimate_000'
      9) Write CSV to `output_path` (or f'{prefix}_tidy.csv' if not provided).
    """

    # ---- helpers ----
    def _find_estimates_sheet(xls: pd.ExcelFile) -> str:
        for s in xls.sheet_names:
            if "estimates" in s.lower() or "estimate" in s.lower():
                return s
        # fallback: use the first sheet if none matched
        return xls.sheet_names[0]

    def _remove_metadata_block(df: pd.DataFrame) -> pd.DataFrame:
        """
        Detect block in first column that starts with '* estimate' and ends with '© Commonwealth of Australia'.
        Remove those rows inclusively. If not found, return df unchanged.
        """
        if df.empty:
            return df
        first_col_name = df.columns[0]
        first_col = df[first_col_name].astype(str).str.strip()

        start_mask = first_col.str.lower().str.startswith("* estimate")
        end_mask = first_col.str.lower().str.contains("commonwealth of australia")

        start_idxs = df.index[start_mask].to_list()
        end_idxs = df.index[end_mask].to_list()
        if not start_idxs or not end_idxs:
            return df

        start_idx = start_idxs[0]
        # choose the last '© Commonwealth...' that occurs after start, else the first one
        end_idx_candidates = [i for i in end_idxs if i >= start_idx]
        end_idx = (end_idx_candidates[-1] if end_idx_candidates else end_idxs[0])

        # Drop inclusive block
        return df.drop(index=range(start_idx, end_idx + 1), errors="ignore")

    def _auto_header_row(df: pd.DataFrame) -> int:
        """
        Return the row index that contains the age group labels across columns 1:.
        Heuristics:
          - If a row's first column contains 'Age group' → header is the next row.
          - Else choose the row (within first ~10) with the max non-null count in columns 1:.
        """
        if df.empty:
            return 0
        first_col_name = df.columns[0]

        # Look for "Age group" marker
        col0 = df[first_col_name].astype(str).str.lower()
        marker_rows = df.index[col0.str.contains("age group", na=False)].to_list()
        if marker_rows:
            candidate = marker_rows[0] + 1
            if candidate < len(df):
                return candidate

        # Fallback: densest non-null row across cols 1: within first 10 rows
        lookahead = min(10, len(df))
        sub = df.iloc[:lookahead, 1:]
        nn = sub.notna().sum(axis=1)
        return int(nn.idxmax())

    def _strip_footnote_markers(df: pd.DataFrame) -> pd.DataFrame:
        """Remove footnote markers like (a) .. (h) from all string cells."""
        footnote_re = re.compile(r"\(([a-hA-H])\)")
        def clean_cell(x):
            if isinstance(x, str):
                return footnote_re.sub("", x).strip()
            return x
        return df.map(clean_cell)
    
    def _strip_footnote_markers_list(values: List[str]) -> List[str]:
        """Remove footnote markers like (a) .. (h) from a list of strings."""
        footnote_re = re.compile(r"\(([a-hA-H])\)")
        return [footnote_re.sub("", str(v)).strip() for v in values]

    # ---- load ----
    xls = pd.ExcelFile(file_path)
    sheet = _find_estimates_sheet(xls)
    df = pd.read_excel(xls, sheet_name=sheet, header=None)  # no header yet; we’ll set it
    # Drop fully empty rows/cols early
    df = df.dropna(how="all").dropna(axis=1, how="all")

    # ---- remove metadata block by content, not index ----
    df = _remove_metadata_block(df)
    df = df.dropna(how="all").dropna(axis=1, how="all")  # re-trim

    # ---- detect and promote header row ----
    header_row_idx = _auto_header_row(df)
    
    # Extract and clean age group labels
    age_labels_raw = df.iloc[header_row_idx, 1:].fillna("").tolist()
    age_labels = _strip_footnote_markers_list(age_labels_raw)

    # Set cleaned column names
    new_cols = [f"{prefix}_row_label"] + age_labels
    df.columns = new_cols

    # Drop rows up through the header row (they’re not data rows)
    df = df.iloc[header_row_idx + 1 :].reset_index(drop=True)

    # Strip footnote markers in all cells (e.g., "(a)".."(h)")
    df = _strip_footnote_markers(df)

    # Ensure no fully empty columns remain
    df = df.dropna(axis=1, how="all")
    
    # ---- identify structure rows ----
    age_cols: List[str] = [c for c in df.columns if c != f"{prefix}_row_label"]
    # A "header row" has all NaN across age columns
    is_header_row = df[age_cols].isna().all(axis=1)

    # Sex section headers
    sex_values = {"persons", "males", "females", "male", "female"}  # a bit more tolerant
    rl_lower = df[f"{prefix}_row_label"].astype(str).str.strip().str.lower()
    is_sex_header = is_header_row & rl_lower.isin(sex_values)

    # Category-group headers: header rows that are not sex headers
    is_group_header = is_header_row & ~is_sex_header

    # ---- forward-fill sex and category group ----
    df[f"{prefix}_sex"] = pd.NA
    df.loc[is_sex_header, f"{prefix}_sex"] = df.loc[is_sex_header, f"{prefix}_row_label"]
    df[f"{prefix}_sex"] = df[f"{prefix}_sex"].ffill()

    df[f"{prefix}_category_group"] = pd.NA
    df.loc[is_group_header, f"{prefix}_category_group"] = df.loc[is_group_header, f"{prefix}_row_label"]
    df[f"{prefix}_category_group"] = df[f"{prefix}_category_group"].ffill()

    # ---- keep only data rows (not structural headers) ----
    data = df.loc[~is_header_row].copy()
    data = data.rename(columns={f"{prefix}_row_label": f"{prefix}_category"})

    # ---- melt to tidy ----
    tidy = data.melt(
        id_vars=[f"{prefix}_sex", f"{prefix}_category_group", f"{prefix}_category"],
        value_vars=age_cols,
        var_name=f"{prefix}_age_group",
        value_name=f"{prefix}_estimate_000"
    )

    # clean values
    tidy[f"{prefix}_estimate_000"] = pd.to_numeric(tidy[f"{prefix}_estimate_000"], errors="coerce")
    tidy = tidy.dropna(subset=[f"{prefix}_estimate_000"]).reset_index(drop=True)

    # order columns
    tidy = tidy[
        [f"{prefix}_sex", f"{prefix}_category_group", f"{prefix}_category",
         f"{prefix}_age_group", f"{prefix}_estimate_000"]]
    
        # ---- special handling for smoking table ----
        # ---- special handling for smoking table (fix only the "Current smoker" group) ----
    if prefix == "smoke":
        sex_col = f"{prefix}_sex"
        grp_col = f"{prefix}_category_group"
        cat_col = f"{prefix}_category"
        age_col = f"{prefix}_age_group"
        est_col = f"{prefix}_estimate_000"

        # normalized helpers
        grp_norm = tidy[grp_col].astype(str).str.strip().str.lower()
        cat_norm = (
            tidy[cat_col].astype(str).str.strip().str.lower()
            .str.replace(",", "", regex=False)
            .str.replace(r"\s+", " ", regex=True)
        )

        # rows that are currently under the "Current smoker" group
        in_current_group = grp_norm.eq("current smoker")

        # (A) Build "Current smoker" category from "Total current smoker(c)" rows
        mask_total_current = in_current_group & cat_norm.str.startswith("total current smoker")
        current_rows = tidy.loc[mask_total_current, [sex_col, age_col, est_col]].copy()
        current_rows[grp_col] = "Smoker status"
        current_rows[cat_col] = "Current smoker"

        # (B) Keep the other three categories from the same group
        keep_map = {
            "ex-smoker": "Ex-smoker",
            "never smoked": "Never smoked",
            "total persons aged 15 years and over": "Total persons aged 15 years and over",
            "total persons 15 years and over": "Total persons aged 15 years and over",  # tolerate punctuation variants
        }
        mask_keep_three = in_current_group & cat_norm.isin(keep_map.keys())
        keep_rows = tidy.loc[mask_keep_three, [sex_col, age_col, est_col]].copy()
        # set target casing and group name
        keep_rows[cat_col] = cat_norm[mask_keep_three].map(keep_map)
        keep_rows[grp_col] = "Smoker status"

        # (C) All other rows (NOT in the "Current smoker" group) stay as-is
        other_rows = tidy.loc[~in_current_group].copy()

        # (D) Concatenate back together
        tidy = pd.concat([other_rows, current_rows, keep_rows], ignore_index=True)

        # Reorder columns just in case
        tidy = tidy[[sex_col, grp_col, cat_col, age_col, est_col]]

    # ---- write CSV ----
    if output_path is None:
        output_path = f"{prefix}_tidy.csv"
    tidy.to_csv(output_path, index=False)
    return output_path

In [9]:
csv_path = process_estimates_file("datasets/Alcohol_Consumption.xlsx", prefix="alco")
print("Wrote:", csv_path)

csv_path = process_estimates_file("datasets/Smoking_and_Vaping.xlsx", prefix="smoke")
print("Wrote:", csv_path)

# csv_path = process_estimates_file("datasets/BMI.xlsx", prefix="bmi")
# print("Wrote:", csv_path)

# csv_path = process_estimates_file("datasets/Dietary_Behaviour.xlsx", prefix="diet")
# print("Wrote:", csv_path)

Wrote: alco_tidy.csv
Wrote: smoke_tidy.csv
